In [ ]:
from langchain import PromptTemplate,  LLMChain
from langchain.chat_models import ChatOpenAI
import pandas as pd
import re
import csv

In [ ]:
def gpt4_ranking(row):
    column_names = row.keys().tolist()
    output_column_names = column_names[2:]
    
    Question=row["Question"]
    GT_Answer=row["GT_Answer"]
    # SKnowGPT_gpt3=row["SKnowGPT_gpt3"]
    # SKnowGPT_mixtral=row["SKnowGPT_mixtral"]
    # SKnowGPT_llama3=row["SKnowGPT_llama3"]
    # gpt3=row["gpt3"]
    # Mixtral=row["Mixtral"]
    # Llama3=row["Llama3"]

    SKnowGPT_gpt3=row["SKnowGPT_gpt3"]
    Mindmap=row["Mindmap"]
    gpt3=row["gpt3"]
    Only_KG=row["Only_KG"]

    
    template = """
    Question: {Question}
    
    Reference: {GT_Answer}
    
    Outputs:
    Output 1: {SKnowGPT_gpt3}
    Output 2: {Mindmap}
    Output 3: {gpt3}
    Output 4: {Only_KG}
    
    Task: Rank the outputs based on their factual match to the disease, drug, and test recommendations relevant to the question and provided in the reference answer, from highest to lowest.

    Output Format:

    1. Output1
    2. Output4
    3. Output2
    4. Output3
    """

    prompt = PromptTemplate(
        template = template,
        input_variables = output_column_names
    )

    llm_chain = LLMChain(prompt=prompt, llm=llm)
    # llm_response = llm_chain.run(Question=Question,GT_Answer=GT_Answer, SKnowGPT_mixtral=SKnowGPT_mixtral, SKnowGPT_llama3=SKnowGPT_llama3, SKnowGPT_gpt3=SKnowGPT_gpt3, Mixtral=Mixtral, Llama3=Llama3, gpt3=gpt3)
    
    llm_response = llm_chain.run(Question=Question,GT_Answer=GT_Answer, Mindmap=Mindmap, SKnowGPT_gpt3=SKnowGPT_gpt3, Only_KG=Only_KG, gpt3=gpt3)

    return llm_response


In [ ]:
openai_api_key = "enter_your_openai_api_key"

llm = ChatOpenAI(
    model="gpt-4",
    api_key=openai_api_key,
    temperature=0.0,
    request_timeout=30,
    max_retries=3,
    timeout=60 * 3,
)

# Read the input CSV file 
# input_file = './GPT4_Ranking_results/GenMedGPT_SKnowGPT_vs_basellms_gpt4_ranking_input.csv'
input_file = './GPT4_Ranking_results/GenMedGPT_SKnowGPT_vs_baseline_gpt4_ranking_input.csv'
df = pd.read_csv(input_file, encoding='ISO 8859-1')

ranking = []

for i, row in df.iterrows():
    print("---",i+1,"---")
    # gpt4_ranking(row)
    llm_response = gpt4_ranking(row)
    ranking.append(llm_response)

df['GPT_ranking'] = ranking

# Write the output to a new CSV file
# output_file = './GPT4_Ranking_results/GenMedGPT_SKnowGPT_vs_basellms_gpt4_ranking_output.csv'
output_file = './GPT4_Ranking_results/GenMedGPT_SKnowGPT_vs_baseline_table1_gpt4_ranking_output.csv'
df.to_csv(output_file, index=False)


In [ ]:
input_file = './GPT4_Ranking_results/GenMedGPT_SKnowGPT_vs_baseline_table1_gpt4_ranking_output.csv'
output_file = './GPT4_Ranking_results/GenMedGPT_SKnowGPT_vs_baseline_table1_gpt4_ranking_output_computed.csv'

with open(input_file,'r',newline="") as f_input, open(output_file, 'a+', newline='') as f_output:
  reader = csv.reader(f_input)
  writer = csv.writer(f_output)

  header = next(reader)
  # header.extend(["output1_ranking","output2_ranking","output3_ranking","output4_ranking","output5_ranking","output6_ranking"])
  header.extend(["output1_ranking","output2_ranking","output3_ranking","output4_ranking"])
  writer.writerow(header)
  j = 0
  matches = []
  for i, row in enumerate(reader):
    # print(i)
    # print(i, row)
    j += 1
    # output_text = row[9].lower()
    output_text = row[7].lower()
    print(output_text)
    if len(matches) != 12:
      # print("1st match ")
      matches = re.findall(r'\d+\.\s+(output\s*\d+)', output_text)
    
    rankings = {}
    
    for i, match in enumerate(matches):
      rank = i + 1
      output = match.replace(" ","")
      rankings[output] = rank

    # print("output = ",output)
    # print("rank = ",rank)

    output_dic = {}

    for output, rank in sorted(rankings.items(), key=lambda x: x[1]):
      output_dic[output]=rank
    
    # print("output_dic: ", output_dic)
    try:
      # row.extend([output_dic['output1'],output_dic['output2'],output_dic['output3'],output_dic['output4'],output_dic['output5'],output_dic['output6']])
      row.extend([output_dic['output1'],output_dic['output2'],output_dic['output3'],output_dic['output4']])
      print("FINAL ROW = ",row)
    except:
      # row.extend([0,0,0,0,0,0])
      row.extend([0,0,0,0])
      continue
    writer.writerow(row)

In [ ]:
def calculate_column_average(file_path, column_name):
    df = pd.read_csv(file_path)
    average = df[column_name].mean()
    return average

# column =["output1_ranking","output2_ranking","output3_ranking","output4_ranking","output5_ranking","output6_ranking"]
column =["output1_ranking","output2_ranking","output3_ranking","output4_ranking"]

file_path = './GPT4_Ranking_results/GenMedGPT_SKnowGPT_vs_baseline_table1_gpt4_ranking_output_computed.csv'
for column_name in column:
  average = calculate_column_average(file_path, column_name)
  print(f"The average of column {column_name} is: {round(average, 2)}")